<a href="https://colab.research.google.com/github/ryzary/geospatial-data/blob/main/land_cover_kalimantan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ryza Rynazal (17B00107)

# Investigating Land-cover Changes in West Kalimantan, Indonesia using NDVI

Kalimantan has been experiencing deforestation in the past decades. It is mainly caused by palm oil plantations, illegal logging and forest fires. This notebook is aimed to examine land-cover changes in West Kalimantan using data from Landsat satellites. Here, I used Google Earth Engine with Python API.

Some of the codes were obtained from the course material provided by Prof. Alvin Varquez in the Geospatial Data Analysis Class, Tokyo Institute of Technology. 
<br>
I also found very useful tutorials from https://towardsdatascience.com/@hagi.willy

In [1]:
import ee
import folium

from datetime import datetime as dt
from IPython.display import Image

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=6hnBApfePDEo3ompSAI82XxaQY0PcFq9ST4JVFMxNG0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/5wFJ7PcihdR4VapcnPvYmwyyjIRwAn782C9ZXzD6rLPvQQfRJwjNLoo

Successfully saved authorization token.


In [3]:
ee.Initialize()

## Select the region

In [4]:
#West Kalimantan, Indonesia
my_map = folium.Map(location=[1.5439,109.208],zoom_start=11,width='80%')
my_map

# Landsat 8 Data

### USGS Landsat 8 Surface Reflectance Tier 1

In [5]:
#Landsat 8 
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterDate('2013-01-01','2014-12-31')

#area of interest:
AOI = ee.Geometry.Rectangle([109.308,1.56,109.150,1.540])

landsat8_AOI = landsat8.filterBounds(AOI)

#lowest cloud coverage
L8_lowest_cloud = ee.Image(landsat8_AOI.sort('CLOUD_COVER').first())

#how cloudy is it?
print('Cloud Cover (%)', L8_lowest_cloud.get('CLOUD_COVER').getInfo())

#when was this image with the least cloud cover taken?
date = ee.Date(L8_lowest_cloud.get('system:time_start'))
time = date.getInfo()['value']/1000.
print('Time: ',dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'))

Cloud Cover (%) 5.37
Time:  2013-06-22 03:00:00


# Landsat 5 Data

### USGS Landsat 5 Surface Reflectance Tier 1

In [6]:
landsat5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").filterDate('1985-01-01','2012-12-31')
AOI = ee.Geometry.Rectangle([109.308,1.56,109.150,1.540])
#filter area:
landsat5_AOI = landsat5.filterBounds(AOI)

#the least cloudy image
L5_lowest_cloud = ee.Image(landsat5_AOI.sort('CLOUD_COVER').first())

#how cloudy is it?
print('Cloud Cover (%)', L5_lowest_cloud.get('CLOUD_COVER').getInfo())

#when was this image with the least cloud cover taken?
date = ee.Date(L5_lowest_cloud.get('system:time_start'))
time = date.getInfo()['value']/1000.
print('Time: ',dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'))


Cloud Cover (%) 1
Time:  1992-08-31 02:20:24


# NDVI

- Normalized Difference Vegetation Index, commonly used to examine land cover 
- NDVI = (NIR-VIS)/(NIR+VIS)
- It ranges from -1 to 1 or 0 to 1. (-1 - 0: water&cloud (red); 1: dense vegetation (green))

In [7]:
#Landsat8 : B5 and B4
ndvi_L8 = L8_lowest_cloud.normalizedDifference(['B5','B4'])

#Landsat 5: (B4 and B3)
ndvi_L5 = L5_lowest_cloud.normalizedDifference(['B4','B3'])

ndvi_parameters = {'min':0,'max':1,
                  'dimensions': 512, 'palette': ['red','yellow','green']}

# Interactive map

To switch the map between Landsat8 and Landsat5, use the Layer Control at the top right corner of the map.

In [8]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

final_map = folium.Map(location=[1.5439,109.208],zoom_start=11,width='80%')
final_map.add_ee_layer(ndvi_L5,ndvi_parameters,'Landsat5/1992')
final_map.add_ee_layer(ndvi_L8,ndvi_parameters,'Landsat8/2013')
final_map.add_child(folium.LayerControl())